In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout
from keras.layers import Bidirectional
import re

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/My Drive/IR Project/Labelled_Total_Data.xlsx')
data = data[['Tweet','Label']]
data


,Tweet,Label
0,"people professing nationalism, patriotism and...",-1.0
1,this is some kind of serious conspiracy plz d...,1.0
2,support with caa and nrc,1.0
3,"i understand your emotion, but feel that firi...",0.0
4,rt most of the muslim who led the riot were b...,1.0
...,...,...
4057,can anyone tell the name of the journalist an...,0.0
4058,none can enter india without a visa issued by...,1.0
4059,"rt right or left, wing are meant to keep u af...",0.0
4060,"right or left, wing are meant to keep u afloa...",0.0


In [9]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, split=' ')

tokenizer.fit_on_texts(data['Tweet'].values)
X = tokenizer.texts_to_sequences(data['Tweet'].values)
X = pad_sequences(X)
print("Dimension of data after text text_to_sequence conversion: ", X.shape)

Dimension of data after text text_to_sequence conversion:  (4062, 72)


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
def count_vectorizer(data):
  tweet_data = data['Tweet']
  bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

  # bag-of-words feature matrix
  bow = bow_vectorizer.fit_transform(tweet_data)

  df_bow = pd.DataFrame(bow.todense())

  return df_bow
X = count_vectorizer(data)

In [6]:
from sklearn.model_selection import train_test_split
# Y = pd.get_dummies(data['Label']).values

def split_train_test(X, Y):
  Y=data["Label"].values
  print(Y)
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)
  print("Dimension of training features and label: ", X_train.shape,Y_train.shape)
  print("Dimension of testing features and label: ", X_test.shape,Y_test.shape)

  return X_train, X_test, Y_train, Y_test

Y = data['Label']
X_train, X_test, Y_train, Y_test = split_train_test(X, Y)

[-1.  1.  1. ...  0.  0.  0.]
Dimension of training features and label:  (3249, 1000) (3249,)
Dimension of testing features and label:  (813, 1000) (813,)


In [22]:
print(X_train.shape)
print(X_test.shape)

(3249, 72)
(813, 72)


In [0]:
print(X_test.shape)

(2, 76)


In [0]:
print(X_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(1000, embed_dim, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation = 'softmax'))
model.add(Dropout(0.50))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 128)         128000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6304      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 389,203
Trainable params: 389,203
Non-trainable params: 0
________________________________________________

In [11]:
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values
print(Y_train)
print(Y_test)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

[[0 1 0]
 [1 0 0]
 [0 1 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 1 0]
 [1 0 0]]
(3249, 1000) (3249, 3)
(813, 1000) (813, 3)


In [12]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
 - 292s - loss: 1.0617 - accuracy: 0.4429
Epoch 2/10
 - 293s - loss: 1.0504 - accuracy: 0.4531
Epoch 3/10
 - 292s - loss: 1.0534 - accuracy: 0.4577
Epoch 4/10
 - 290s - loss: 1.0475 - accuracy: 0.4561
Epoch 5/10
 - 283s - loss: 1.0497 - accuracy: 0.4586
Epoch 6/10
 - 282s - loss: 1.0468 - accuracy: 0.4580
Epoch 7/10
 - 283s - loss: 1.0466 - accuracy: 0.4580
Epoch 8/10
 - 283s - loss: 1.0467 - accuracy: 0.4580
Epoch 9/10
 - 286s - loss: 1.0459 - accuracy: 0.4580
Epoch 10/10
 - 288s - loss: 1.0447 - accuracy: 0.4580


In [13]:
validation_size = 100

print(X_test.shape)
print(Y_test.shape)

#X_validate = X_test[-validation_size:]
#Y_validate = Y_test[-validation_size:]
#X_test = X_test[:-validation_size]
#Y_test = Y_test[:-validation_size]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
result = model.predict(X_test)
print("acc: %.2f" % (acc))
print(result)

(813, 1000)
(813, 3)
acc: 0.46
[[0.19632581 0.43993142 0.36374283]
 [0.19632578 0.4399314  0.36374283]
 [0.19632578 0.43993142 0.36374283]
 ...
 [0.1963258  0.43993142 0.36374283]
 [0.19632578 0.4399314  0.36374283]
 [0.19632578 0.4399314  0.36374283]]


In [16]:
print(type(result))
print(result.shape)

<class 'numpy.ndarray'>
(813, 3)


In [21]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
test_predictions = []
corr_count = 0
for x in range(len(X_test)):
    
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    max_index1 = np.argmax(result)
    max_index2 = np.argmax(Y_test[x])
    if max_index1 == max_index2:
      corr_count += 1

    
    if max_index1==0:
      test_predictions.append(-1)
    elif max_index1==1:
      test_predictions.append(0)
    else:
      test_predictions.append(1)
    
    if np.argmax(result) == np.argmax(Y_test[x]):
        if np.argmax(Y_test[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_test[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

accuracy = corr_count/len(Y_test)
print("Accuracy of the RNN model: ", accuracy)


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

Accuracy of the RNN model:  0.5990159901599016
pos_acc 62.14177978883861 %
neg_acc 50.0 %
